In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sn
import os
from sklearn.preprocessing import OneHotEncoder
import nltk

# Extracting recipes from csv

In [2]:
#users = pd.read_csv('../Data/PP_users.csv')
#recipes = pd.read_csv('../Data/PP_recipes.csv')
raw_interactions = pd.read_csv('./RAW_interactions.csv')
raw_recipes = pd.read_csv('./RAW_recipes.csv')

In [3]:
print(raw_interactions.shape)
raw_interactions.head()

(1132367, 5)


,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [4]:
print(raw_recipes.shape)
raw_recipes.head()

(231637, 12)


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [5]:
# merge recipes and interactions tables

dfinal = raw_interactions.merge(raw_recipes, how='inner', left_on='recipe_id', right_on='id')
print(dfinal.shape)
dfinal.head()

(1132367, 17)


,user_id,recipe_id,date,rating,review,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,40893,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,44394,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,85009,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


### Filtering data
- User that has reviewed >= 100 reviews
- Recipes that has >= 100 reviews

In [6]:
counts_col2 = dfinal.groupby("user_id")["user_id"].transform(len)
counts_col3 = dfinal.groupby("recipe_id")["recipe_id"].transform(len)

mask = (counts_col2 >= 100) & (counts_col3 >= 100)

dfinal = dfinal[mask]

In [7]:
dfinal.drop(columns=['id','submitted'],inplace=True)
print(dfinal.shape)
dfinal.head()

(39398, 15)


,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
1479,327989,9054,2007-02-08,3,I liked this recipe but there are no measureme...,leftover mashed potato pancakes,35,8381,"['60-minutes-or-less', 'time-to-make', 'course...","[266.9, 5.0, 13.0, 27.0, 17.0, 7.0, 16.0]",7,"['mix mashed potatoes , egg , flour , salt , p...",this recipe was recently submitted to a news g...,"['mashed potatoes', 'egg', 'flour', 'salt', 'p...",11
1480,240552,9054,2007-11-06,5,These are great! I had some mashed potatoes f...,leftover mashed potato pancakes,35,8381,"['60-minutes-or-less', 'time-to-make', 'course...","[266.9, 5.0, 13.0, 27.0, 17.0, 7.0, 16.0]",7,"['mix mashed potatoes , egg , flour , salt , p...",this recipe was recently submitted to a news g...,"['mashed potatoes', 'egg', 'flour', 'salt', 'p...",11
1481,206747,9054,2007-11-07,4,I commented on another recipe here -- if the l...,leftover mashed potato pancakes,35,8381,"['60-minutes-or-less', 'time-to-make', 'course...","[266.9, 5.0, 13.0, 27.0, 17.0, 7.0, 16.0]",7,"['mix mashed potatoes , egg , flour , salt , p...",this recipe was recently submitted to a news g...,"['mashed potatoes', 'egg', 'flour', 'salt', 'p...",11
1487,936601,9054,2008-09-08,5,Great way to use left over mashed potatoes. I...,leftover mashed potato pancakes,35,8381,"['60-minutes-or-less', 'time-to-make', 'course...","[266.9, 5.0, 13.0, 27.0, 17.0, 7.0, 16.0]",7,"['mix mashed potatoes , egg , flour , salt , p...",this recipe was recently submitted to a news g...,"['mashed potatoes', 'egg', 'flour', 'salt', 'p...",11
1489,485151,9054,2009-01-04,5,Oh yummy! These were just like the ones I had ...,leftover mashed potato pancakes,35,8381,"['60-minutes-or-less', 'time-to-make', 'course...","[266.9, 5.0, 13.0, 27.0, 17.0, 7.0, 16.0]",7,"['mix mashed potatoes , egg , flour , salt , p...",this recipe was recently submitted to a news g...,"['mashed potatoes', 'egg', 'flour', 'salt', 'p...",11


# Change date format (from str to date format)

In [8]:
from datetime import datetime

dfinal['date'] = dfinal['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date())

In [9]:
print(type(dfinal['date'].iloc[0]))

<class 'datetime.date'>


# Extract nutritional Components

- from string to respective categories
- convert them to float type

In [10]:
df_nutri = dfinal[['recipe_id', 'name','n_steps','n_ingredients','minutes','nutrition']]
df_nutri.drop_duplicates(inplace=True)
df_nutri.reset_index(drop=True,inplace=True)
df_nutri

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,recipe_id,name,n_steps,n_ingredients,minutes,nutrition
0,9054,leftover mashed potato pancakes,7,11,35,"[266.9, 5.0, 13.0, 27.0, 17.0, 7.0, 16.0]"
1,135585,baked buffalo chicken breasts,7,7,40,"[228.7, 20.0, 1.0, 20.0, 48.0, 18.0, 0.0]"
2,93886,crock pot apple butter,11,5,805,"[154.2, 0.0, 145.0, 0.0, 0.0, 0.0, 13.0]"
3,96886,kittencal s spinach parmesan rice bake,7,13,40,"[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]"
4,50022,the best banana bread,13,10,85,"[528.3, 34.0, 198.0, 23.0, 12.0, 67.0, 26.0]"
...,...,...,...,...,...,...
780,47391,crispy tilapia,9,11,16,"[507.3, 12.0, 6.0, 18.0, 149.0, 13.0, 11.0]"
781,131018,pulled pork crock pot,26,4,1110,"[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]"
782,57679,sandra s key lime pie,12,5,30,"[426.2, 35.0, 158.0, 13.0, 15.0, 51.0, 16.0]"
783,219251,slow cooker macaroni cheese,7,10,185,"[273.9, 27.0, 8.0, 20.0, 23.0, 53.0, 5.0]"


In [11]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri.nutrition.str.split(",",expand=True)

C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
df_nutri['calories'] =  df_nutri['calories'].apply(lambda x: x.replace('[','')) 
df_nutri['carbohydrates (PDV)'] =  df_nutri['carbohydrates (PDV)'].apply(lambda x: x.replace(']',''))

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']] = df_nutri[['calories','total fat (PDV)','sugar (PDV)','sodium (PDV)','protein (PDV)','saturated fat (PDV)','carbohydrates (PDV)']].astype('float')

In [14]:
print(df_nutri.shape)
df_nutri.drop(['nutrition'], axis=1,inplace=True)
df_nutri.head()

(785, 13)


C:\Users\Gi Han\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,9054,leftover mashed potato pancakes,7,11,35,266.9,5.0,13.0,27.0,17.0,7.0,16.0
1,135585,baked buffalo chicken breasts,7,7,40,228.7,20.0,1.0,20.0,48.0,18.0,0.0
2,93886,crock pot apple butter,11,5,805,154.2,0.0,145.0,0.0,0.0,0.0,13.0
3,96886,kittencal s spinach parmesan rice bake,7,13,40,351.2,36.0,3.0,20.0,35.0,70.0,5.0
4,50022,the best banana bread,13,10,85,528.3,34.0,198.0,23.0,12.0,67.0,26.0


# Combine dataset 
Merge with user information

In [15]:
dfuser = dfinal[['user_id', 'date', 'review','rating','recipe_id']]
dfuser.drop_duplicates(inplace=True)
dfuser.reset_index(drop=True,inplace=True)
dfuser

C:\Users\Gi Han\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user_id,date,review,rating,recipe_id
0,327989,2007-02-08,I liked this recipe but there are no measureme...,3,9054
1,240552,2007-11-06,These are great! I had some mashed potatoes f...,5,9054
2,206747,2007-11-07,I commented on another recipe here -- if the l...,4,9054
3,936601,2008-09-08,Great way to use left over mashed potatoes. I...,5,9054
4,485151,2009-01-04,Oh yummy! These were just like the ones I had ...,5,9054
...,...,...,...,...,...
39393,319738,2013-05-11,I have made cakes from scratch for years and t...,3,38966
39394,364211,2013-11-15,Made this for my son&#039;s birthday today. Qu...,4,38966
39395,500860,2014-03-10,"Very quick, very moist. Excellent with a good ...",4,38966
39396,302961,2014-07-21,"I have never left a one star review before, I ...",1,38966


In [16]:
df_combine = df_nutri.merge(dfuser, how='inner', on='recipe_id')
df_combine

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
0,9054,leftover mashed potato pancakes,7,11,35,266.9,5.0,13.0,27.0,17.0,7.0,16.0,327989,2007-02-08,I liked this recipe but there are no measureme...,3
1,9054,leftover mashed potato pancakes,7,11,35,266.9,5.0,13.0,27.0,17.0,7.0,16.0,240552,2007-11-06,These are great! I had some mashed potatoes f...,5
2,9054,leftover mashed potato pancakes,7,11,35,266.9,5.0,13.0,27.0,17.0,7.0,16.0,206747,2007-11-07,I commented on another recipe here -- if the l...,4
3,9054,leftover mashed potato pancakes,7,11,35,266.9,5.0,13.0,27.0,17.0,7.0,16.0,936601,2008-09-08,Great way to use left over mashed potatoes. I...,5
4,9054,leftover mashed potato pancakes,7,11,35,266.9,5.0,13.0,27.0,17.0,7.0,16.0,485151,2009-01-04,Oh yummy! These were just like the ones I had ...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39393,38966,quick yellow cake,6,8,42,264.7,15.0,89.0,12.0,7.0,14.0,13.0,319738,2013-05-11,I have made cakes from scratch for years and t...,3
39394,38966,quick yellow cake,6,8,42,264.7,15.0,89.0,12.0,7.0,14.0,13.0,364211,2013-11-15,Made this for my son&#039;s birthday today. Qu...,4
39395,38966,quick yellow cake,6,8,42,264.7,15.0,89.0,12.0,7.0,14.0,13.0,500860,2014-03-10,"Very quick, very moist. Excellent with a good ...",4
39396,38966,quick yellow cake,6,8,42,264.7,15.0,89.0,12.0,7.0,14.0,13.0,302961,2014-07-21,"I have never left a one star review before, I ...",1


In [17]:
# sort dataframe by date
final = df_combine.sort_values(by="date")
final.drop_duplicates(inplace=True)
print(final.shape)
final.head()

(39398, 16)


,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
23960,2886,best banana bread,13,8,65,272.8,16.0,97.0,14.0,7.0,31.0,14.0,2312,2001-04-18,I used one more banana than called for because...,5
6844,8782,roast sticky chicken,12,10,325,238.0,24.0,9.0,67.0,35.0,22.0,2.0,7802,2001-05-16,"The recipe was easy to prepare, though a bit m...",4
30097,5478,easy garlic chicken,8,5,30,221.3,9.0,53.0,5.0,50.0,5.0,4.0,11297,2001-07-17,Easy and tasty. Even easier clean-up by lini...,4
2120,8739,wholly guacamole,13,8,20,500.7,68.0,14.0,0.0,13.0,32.0,10.0,4470,2001-07-24,I don't know if I enjoyed the quacamole or the...,5
2121,8739,wholly guacamole,13,8,20,500.7,68.0,14.0,0.0,13.0,32.0,10.0,9869,2001-09-07,"The chips were warm, the beer was cold, the gu...",5


In [18]:
# Convert to csv
final.to_csv('final_dataset.csv', index=False)

# Train & Test Dataset

In [19]:
train_set, test_set= np.split(final, [int(.8 * len(final))])

In [20]:
train_set

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
23960,2886,best banana bread,13,8,65,272.8,16.0,97.0,14.0,7.0,31.0,14.0,2312,2001-04-18,I used one more banana than called for because...,5
6844,8782,roast sticky chicken,12,10,325,238.0,24.0,9.0,67.0,35.0,22.0,2.0,7802,2001-05-16,"The recipe was easy to prepare, though a bit m...",4
30097,5478,easy garlic chicken,8,5,30,221.3,9.0,53.0,5.0,50.0,5.0,4.0,11297,2001-07-17,Easy and tasty. Even easier clean-up by lini...,4
2120,8739,wholly guacamole,13,8,20,500.7,68.0,14.0,0.0,13.0,32.0,10.0,4470,2001-07-24,I don't know if I enjoyed the quacamole or the...,5
2121,8739,wholly guacamole,13,8,20,500.7,68.0,14.0,0.0,13.0,32.0,10.0,9869,2001-09-07,"The chips were warm, the beer was cold, the gu...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8547,208583,kittencal s juicy hamburger burger,9,8,10,351.2,36.0,1.0,13.0,59.0,45.0,1.0,1320105,2010-05-22,Great flavor and very easy! I didn't read far...,5
3722,126623,jolean s cheese potato smoked sausage casserole,11,10,60,871.4,100.0,26.0,96.0,67.0,158.0,12.0,41218,2010-05-22,This is good ole comfort food. I followed the...,4
3527,9240,chocolate eclair dessert,11,5,20,480.1,31.0,175.0,14.0,12.0,48.0,23.0,124190,2010-05-22,Very well received at the pitch-in today. Made...,5
4706,137364,perfect basic white rice,13,5,20,168.8,0.0,0.0,24.0,6.0,0.0,12.0,705251,2010-05-22,I am in shock that one step makes a whole worl...,5


In [21]:
test_set

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
16403,29251,jalapeno popper burgers,8,10,25,397.6,45.0,5.0,16.0,58.0,62.0,0.0,1320105,2010-05-22,Very good burger! I don't really think it tas...,5
3211,27210,best grilled pork chops,8,6,135,597.5,62.0,1.0,44.0,107.0,53.0,0.0,33159,2010-05-22,Really good - loads of flavour!,4
15361,34335,herbed pork tenderloin,6,12,50,225.2,16.0,10.0,14.0,48.0,11.0,2.0,11176,2010-05-22,This is SO easy and SO good... a great recipe ...,5
4705,137364,perfect basic white rice,13,5,20,168.8,0.0,0.0,24.0,6.0,0.0,12.0,195589,2010-05-22,Very nice! I have always cheated and used inst...,5
4704,137364,perfect basic white rice,13,5,20,168.8,0.0,0.0,24.0,6.0,0.0,12.0,526666,2010-05-22,"Simple and easy, this a GREAT! I added the sal...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19892,187362,light and fluffy pumpkin pancakes,7,12,16,273.0,12.0,41.0,25.0,16.0,23.0,14.0,198154,2018-12-05,I had homemade pumpkin pur&eacute;e instead of...,4
32762,69173,kittencal s italian melt in your mouth meatballs,5,10,50,1312.6,129.0,8.0,108.0,214.0,174.0,8.0,315055,2018-12-09,This is our go to meatball recipe. Turns out w...,5
27469,80413,homemade hamburger buns bread machine,9,7,19,198.9,5.0,17.0,7.0,11.0,9.0,11.0,400708,2018-12-12,"OMG, these turned out sooooo good and so much ...",5
11467,67930,spiced pumpkin waffles,8,12,15,227.2,14.0,31.0,16.0,12.0,26.0,10.0,198154,2018-12-13,I used fresh pumpkin pur&eacute;e instead of c...,3


In [22]:
# convert to csv
train_set.to_csv('train_dataset_all.csv', index=False)
test_set.to_csv('test_dataset_all.csv', index=False)

## Test-validation dataset

In [23]:
val_set = test_set[:len(test_set)//2]
val_set

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
16403,29251,jalapeno popper burgers,8,10,25,397.6,45.0,5.0,16.0,58.0,62.0,0.0,1320105,2010-05-22,Very good burger! I don't really think it tas...,5
3211,27210,best grilled pork chops,8,6,135,597.5,62.0,1.0,44.0,107.0,53.0,0.0,33159,2010-05-22,Really good - loads of flavour!,4
15361,34335,herbed pork tenderloin,6,12,50,225.2,16.0,10.0,14.0,48.0,11.0,2.0,11176,2010-05-22,This is SO easy and SO good... a great recipe ...,5
4705,137364,perfect basic white rice,13,5,20,168.8,0.0,0.0,24.0,6.0,0.0,12.0,195589,2010-05-22,Very nice! I have always cheated and used inst...,5
4704,137364,perfect basic white rice,13,5,20,168.8,0.0,0.0,24.0,6.0,0.0,12.0,526666,2010-05-22,"Simple and easy, this a GREAT! I added the sal...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10740,102335,moist and tender turkey or turkey breast,15,4,130,341.4,29.0,0.0,10.0,78.0,33.0,0.0,366781,2012-04-15,Other than taking longer ( I think my oven is ...,5
22954,35988,tortellini tomato spinach soup,7,10,30,312.6,15.0,21.0,50.0,34.0,16.0,13.0,663997,2012-04-15,Great easy recipe. Added an extra can of toma...,4
19482,43023,creamy garlic penne pasta,7,9,15,363.8,12.0,0.0,9.0,19.0,21.0,21.0,173579,2012-04-15,I had this for a family gathering. Everyone l...,5
1731,11923,hot german potato salad,5,9,30,347.6,6.0,76.0,11.0,16.0,6.0,23.0,80980,2012-04-15,This was the best part of our meal today. I h...,5


In [24]:
test_set2 = test_set[len(test_set)//2:]
test_set2

,recipe_id,name,n_steps,n_ingredients,minutes,calories,total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV),user_id,date,review,rating
11346,54257,yes virginia there is a great meatloaf,15,10,80,493.1,43.0,75.0,52.0,70.0,55.0,7.0,341344,2012-04-16,Made exactly as suggested. Outstanding with v...,5
2807,107072,crispy ginger beef,13,14,30,944.9,47.0,214.0,72.0,117.0,52.0,35.0,176615,2012-04-16,Really good! Definitely a do-over. Everyone en...,5
10668,76491,thick chocolate pudding,11,6,15,160.8,1.0,67.0,6.0,11.0,1.0,10.0,841895,2012-04-16,"I made this in the microwave tonight, scaled d...",5
583,26110,southern buttermilk biscuits,21,6,22,162.6,11.0,5.0,16.0,6.0,22.0,6.0,428060,2012-04-17,These were very good biscuits. They puffed up ...,4
22301,104628,soft italian breadsticks abm,8,9,35,80.5,3.0,4.0,6.0,3.0,6.0,4.0,1242099,2012-04-17,Just great! I used 1/3 whole wheat flour. We...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19892,187362,light and fluffy pumpkin pancakes,7,12,16,273.0,12.0,41.0,25.0,16.0,23.0,14.0,198154,2018-12-05,I had homemade pumpkin pur&eacute;e instead of...,4
32762,69173,kittencal s italian melt in your mouth meatballs,5,10,50,1312.6,129.0,8.0,108.0,214.0,174.0,8.0,315055,2018-12-09,This is our go to meatball recipe. Turns out w...,5
27469,80413,homemade hamburger buns bread machine,9,7,19,198.9,5.0,17.0,7.0,11.0,9.0,11.0,400708,2018-12-12,"OMG, these turned out sooooo good and so much ...",5
11467,67930,spiced pumpkin waffles,8,12,15,227.2,14.0,31.0,16.0,12.0,26.0,10.0,198154,2018-12-13,I used fresh pumpkin pur&eacute;e instead of c...,3


In [25]:
# convert to csv
val_set.to_csv('validation_dataset_all.csv', index=False)
test_set2.to_csv('test_v2_dataset_all.csv', index=False)